Escolher duas doenças no Brasil e plotar a distribuição dos casos entre as duas usando um plot do tipo hex bin

por enquanto:
- Alcohol Use Disorders x Cirrhosis and Other Chronic Liver Diseases

In [13]:
import pandas as pd

from bokeh.models import ColumnDataSource
from bokeh.plotting import output_notebook, show, figure
from bokeh.models import HoverTool

import os
os.chdir("E:/Users/B51995/Desktop/A2_ICD")

output_notebook()

df_cause_of_death = pd.read_csv("new_cause_of_death.csv")

df_alcohol_brasil = df_cause_of_death[df_cause_of_death["Country/Territory"] == "Brazil"].groupby("Year")[["Alcohol Use Disorders", "Cirrhosis and Other Chronic Liver Diseases"]].sum().reset_index()

display(df_alcohol_brasil)

data_alcohol_brasil = ColumnDataSource(df_alcohol_brasil)

plot = figure(match_aspect = True)

plot.title = "Distribuição de disordens de uso de alcool e doenças crônicas do fígado"
# plot.background_fill_color = "#440154"
plot.grid.visible = False

# r, bins = plot.hexbin(x = "Cirrhosis and Other Chronic Liver Diseases", y = "Alcohol Use Disorders", size = 0.5, hover_color = "pink", hover_alpha = 0.8, source = data_alcohol_brasil)

plot.circle(x = "Cirrhosis and Other Chronic Liver Diseases", y = "Alcohol Use Disorders", color = "Black", size = 1, source = data_alcohol_brasil)

# plot.add_tools(HoverTool(
#     tooltips = [("count", "@c")],
#     mode="mouse", point_policy="follow_mouse", renderers=[r]
# ))

show(plot)

Loading BokehJS ...

,Year,Alcohol Use Disorders,Cirrhosis and Other Chronic Liver Diseases
0,1990,4593,24474
1,1991,4749,24488
2,1992,4920,24913
3,1993,5131,25965
4,1994,5329,26329
5,1995,5518,26604
6,1996,5741,26864
7,1997,5969,26991
8,1998,6182,27520
9,1999,6447,27998


In [18]:
df_deaths_brazil = df_cause_of_death[df_cause_of_death["Country/Territory"] == "Brazil"]
data_deaths_brazil = ColumnDataSource(df_deaths_brazil)

plot = figure()
plot.scatter(x = "Year", y = "Cardiovascular Diseases", source = data_deaths_brazil)
plot.scatter(x = "Year", y = "Self-harm", source = data_deaths_brazil)
plot.scatter(x = "Year", y = "Tuberculosis", source = data_deaths_brazil)
plot.scatter(x = "Year", y = "Diarrheal Diseases", source = data_deaths_brazil)
plot.scatter(x = "Year", y = "Fire, Heat, and Hot Substances", source = data_deaths_brazil)

show(plot)

Box plot

Agrupar por país e fazer box plot

In [34]:
import pandas as pd

from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, show
from bokeh.sampledata.autompg2 import autompg2
from bokeh.transform import factor_cmap

output_notebook()

df_mesmo = df_cause_of_death[["Continente", "Cardiovascular Diseases"]].rename(columns = {"Continente" : "kind"})
# df = autompg2[["class", "hwy"]].rename(columns={"class": "kind"})

kinds = df_mesmo.kind.unique()

# compute quantiles
qs = df_mesmo.groupby("kind")["Cardiovascular Diseases"].quantile([0.25, 0.5, 0.75])
qs = qs.unstack().reset_index()
qs.columns = ["kind", "q1", "q2", "q3"]
df_mesmo = pd.merge(df_mesmo, qs, on="kind", how="left")

display(df_mesmo)

# compute IQR outlier bounds
iqr = df_mesmo.q3 - df_mesmo.q1
df_mesmo["upper"] = df_mesmo.q3 + 1.5*iqr
df_mesmo["lower"] = df_mesmo.q1 - 1.5*iqr

df_mesmo_mesmo = df_mesmo.groupby("kind").sum().reset_index()

source = ColumnDataSource(df_mesmo_mesmo)

boxplot = figure(x_range=kinds,
           title="Mortes por doenças cardiovasculares por continente",
           background_fill_color="#eaefef", y_axis_label="Mortes por acidentes cardiovasculares")

# outlier range
whisker = Whisker(base="kind", upper="upper", lower="lower", source=source)
whisker.upper_head.size = whisker.lower_head.size = 20
boxplot.add_layout(whisker)

# quantile boxes
cmap = factor_cmap("kind", "TolRainbow7", kinds)
boxplot.vbar("kind", 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
boxplot.vbar("kind", 0.7, "q1", "q2", source=source, color=cmap, line_color="black")

# outliers
outliers = df_mesmo[~df_mesmo["Cardiovascular Diseases"].between(df_mesmo.lower, df_mesmo.upper)]
boxplot.scatter("kind", "Cardiovascular Diseases", source=outliers, size=6, color="black", alpha=0.3)

boxplot.xgrid.grid_line_color = None
boxplot.axis.major_label_text_font_size="14px"
boxplot.axis.axis_label_text_font_size="12px"

show(boxplot)

Loading BokehJS ...

,kind,Cardiovascular Diseases,q1,q2,q3
0,Asia,44899,6505.00,32495.5,105946.75
1,Asia,45492,6505.00,32495.5,105946.75
2,Asia,46557,6505.00,32495.5,105946.75
3,Asia,47951,6505.00,32495.5,105946.75
4,Asia,49308,6505.00,32495.5,105946.75
...,...,...,...,...,...
6115,Africa,16649,3243.75,9609.0,21493.00
6116,Africa,16937,3243.75,9609.0,21493.00
6117,Africa,17187,3243.75,9609.0,21493.00
6118,Africa,17460,3243.75,9609.0,21493.00
